## Presentation Type Classifier and Networks


### What can You do with this Notebook

- Find and classify imitative points (fugas, imitative duos, and periodic entries)
- Build a network of patterns found in several pieces, and reveal 'communities' of compositions


#### Network Visualization with Louvain Communities

* Read more about **Louvain Community Detection Algorithms**:  https://towardsdatascience.com/community-detection-algorithms-9bd8951e7dae

* **Updated Summer 2022**
* **Key Features**:

    * Uses `getDistance` to identify `close matches` with side-by-side comparison of soggetti.  
    * For example: With a distance of "1", the soggetti `4, 1, 2, 3`, and `5, 1, 2, 3` will count as the same. These are reported as "flexed entries" in a separate column.
    * Finds time intervals between entries (expressed as offsets, like `8.0, 4.0, 8.0`)
    * Finds melodic intervals between first note of successive entries in each pattern (like `P-5, P-8`)
    * Labels Fuga, PEn, and ID according to time intervals between entries. NIm not yet supported!
    * If two entries are separated by more than 10 bars (80 offsets), the tool resets to a new pattern
    * Counts number of entries
    * Provides offset and measure/beat locations
    * Sorts all presentation types by the order in which they appear in the piece
    * Reports voice names of the entries, in order of their appearance
    * Omits singleton soggetti (just one entry of a given motive in isolation)
    
* Read the documentation:  `print(piece.presentationTypes.__doc__)`

In [2]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import numpy as np
import itertools
from itertools import combinations
import networkx as nx
from community import community_louvain
from copy import deepcopy
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

saved_csv folder already exists.


## A.  Start by Finding Presentation Types in a Corpus of Pieces 

* The **CorpusBase** class is a convenient way to find patterns in any given list of pieces.
* The pieces are provided as a **list**, within square brackets and separated by commas.  
* The bracketed list is then contained within the parentheses of `CorpusBase()`
* For example: 

>`prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'
piece_list = ['CRIM_Model_0019',
             'CRIM_Mass_0019_1',
             'CRIM_Mass_0019_2',
             'CRIM_Mass_0019_3',
             'CRIM_Mass_0019_4',
             'CRIM_Mass_0019_5']
suffix = '.mei'
corpus_list  = []
for piece in piece_list:
url = prefix + piece + suffix
    corpus_list.append(url)
corpus = CorpusBase(corpus_list)` 
       
* Read the documentation:  `print(CorpusBase.batch.__doc__)`



In [3]:
prefix = 'https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/'

piece_list = ['CRIM_Model_0019',
             'CRIM_Mass_0019_1']
#              'CRIM_Mass_0019_2',
#              'CRIM_Mass_0019_3',
#              'CRIM_Mass_0019_4',
#              'CRIM_Mass_0019_5']

suffix = '.mei'

In [4]:
import glob
piece_list = []
for name in glob.glob('Music2/*'):
    piece_list.append(name)
corpus = CorpusBase(piece_list)

Successfully imported Music_Files/A_Senfl_ave_from_midi.musicxml
Successfully imported Music_Files/CRIM_Model_0008.mei


### Now Run the PresentationType method on the Corpus

In [5]:

final = pd.DataFrame()

for work in piece_list:
#     file_url = prefix + piece 
#     url = prefix + piece + suffix
    piece = importScore(work)   
    output = piece.presentationTypes(limit_to_entries = True,
                                     head_flex = 1,
                                      body_flex = 0,
                                    include_hidden_types = False,
                                    combine_unisons = False,
                                   melodic_ngram_length = 4)

    final['Validation'] = ""
    final['Comments'] = ""
    final = final.append(output, ignore_index=True)

# final.to_csv("corpus_classified.csv")
# final.to_csv("saved_csv/CRIM_Mass_0019_P_Types_hidden.csv")

Previously imported piece detected.
Previously imported piece detected.


In [6]:
final


,Validation,Comments,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,Parallel_Entries,Parallel_Voice,Count_Non_Overlaps
0,,,Ludwig Senfl,"Ave, maria",0.0,"[1/1.0, 3/1.0, 3/3.0, 6/1.0, 8/1.0, 10/3.0, 12...","[M6, P4, P-5, P5, P-5, P5, P-4, P-12, P15, P-8]","[0.0, 16.0, 20.0, 40.0, 56.0, 76.0, 92.0, 112....","[(5, 1, 2, 2), (4, 1, 2, 2)]","[16.0, 4.0, 20.0, 16.0, 20.0, 16.0, 20.0, 4.0,...","[T2, D2, D1, T2, D1, T2, D1, Ct, B, D1, T1]",FUGA,11.0,True,0.0,None,0.0
1,,,Ludwig Senfl,"Ave, maria",140.0,"[18/3.0, 24/3.0, 28/3.0, 32/1.0, 33/3.0, 36/2....","[M-9, M2, P8, P-12, P15, P-12]","[140.0, 188.0, 220.0, 248.0, 260.0, 282.0, 292.0]","[(4, 1, 1, 2)]","[48.0, 32.0, 28.0, 12.0, 22.0, 10.0]","[D2, B, B, Ct, B, D2, B]",FUGA,7.0,False,0.0,None,2.0
2,,,Ludwig Senfl,"Ave, maria",316.0,"[40/3.0, 41/3.0, 42/1.0, 43/1.0, 44/1.0]","[P1, P-5, P-8, P12]","[316.0, 324.0, 328.0, 336.0, 344.0]","[(-3, -2, 2, 3)]","[8.0, 4.0, 8.0, 8.0]","[D1, Ct, T2, B, D2]",FUGA,5.0,False,0.0,None,0.0
3,,,Ludwig Senfl,"Ave, maria",350.0,"[44/4.0, 46/4.0, 49/2.0, 54/1.0]","[P1, P-5, M2]","[350.0, 366.0, 386.0, 424.0]","[(1, 1, 2, 2)]","[16.0, 20.0, 38.0]","[D1, Ct, T2, Ct]",FUGA,4.0,False,0.0,None,1.0
4,,,Ludwig Senfl,"Ave, maria",450.0,"[57/2.0, 65/1.0, 65/3.0, 67/3.0, 71/1.0, 73/3.0]","[P1, P-5, P8, P-4, P-5]","[450.0, 512.0, 516.0, 532.0, 560.0, 580.0]","[(2, 2, 2, 1), (1, 2, 2, 1)]","[62.0, 4.0, 16.0, 28.0, 20.0]","[D2, D2, T2, D1, Ct, D2]",FUGA,6.0,True,1.0,B,1.0
5,,,Ludwig Senfl,"Ave, maria",456.0,"[58/1.0, 59/1.0, 60/1.0, 61/1.0, 62/4.0]","[P15, P1, P-8, P8]","[456.0, 464.0, 472.0, 480.0, 494.0]","[(-3, -2, 2, 2)]","[8.0, 8.0, 8.0, 14.0]","[B, Ct, D1, T2, D2]",FUGA,5.0,False,0.0,None,0.0
6,,,Ludwig Senfl,"Ave, maria",620.0,"[78/3.0, 82/4.0, 84/3.0]","[P-4, P4]","[620.0, 654.0, 668.0]","[(1, 1, 2, -2)]","[34.0, 14.0]","[D1, D2, D1]",FUGA,3.0,False,1.0,Ct,0.0
7,,,Ludwig Senfl,"Ave, maria",715.0,"[90/2.5, 92/2.5]",[P1],"[715.0, 731.0]","[(-6, 2, 2, 3), (-5, 2, 2, 3)]",[16.0],"[Ct, Ct]",FUGA,2.0,True,0.0,None,1.0
8,,,Ludwig Senfl,"Ave, maria",738.0,"[93/2.0, 94/2.0]",[P8],"[738.0, 746.0]","[(1, 2, 2, -2), (2, 2, 2, -2)]",[8.0],"[T2, D1]",FUGA,2.0,True,0.0,None,0.0
9,,,Ludwig Senfl,"Ave, maria",846.0,"[106/4.0, 107/4.0]",[P1],"[846.0, 854.0]","[(2, 2, 2, 1), (1, 2, 2, 1)]",[8.0],"[D2, D1]",FUGA,2.0,True,0.0,None,0.0


#### Various Options to Filter

* by type
* by piece
* according to length between entries

In [69]:
# filter for only ID or PEN
pen_id_filter = final['Presentation_Type'].isin(["ID", "PEN"])
pen_id_only = final[pen_id_filter]
final = pen_id_only.copy()
final



,Validation,Comments,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,Voices,Presentation_Type,Number_Entries,Flexed_Entries,Parallel_Entries,Parallel_Voice,Count_Non_Overlaps
0,,,Ludwig Senfl,"Ave, maria",0.0,"[1/1.0, 8/1.0, 12/3.0, 19/3.0]","[M9, P1, P1]","[0.0, 56.0, 92.0, 148.0]","[(5, 1, 2, 2), (4, 1, 2, 2)]","[56.0, 36.0, 56.0]","[T2, D1, D1, D1]",ID,4.0,True,0.0,None,3.0
1,,,Ludwig Senfl,"Ave, maria",0.0,"[1/1.0, 3/3.0, 6/1.0]","[M9, P-5]","[0.0, 20.0, 40.0]","[(5, 1, 2, 2), (4, 1, 2, 2)]","[20.0, 20.0]","[T2, D1, T2]",PEN,3.0,True,0.0,None,0.0
2,,,Ludwig Senfl,"Ave, maria",0.0,"[1/1.0, 6/1.0, 15/1.0, 20/1.0]","[P5, M2, P-5]","[0.0, 40.0, 112.0, 152.0]","[(5, 1, 2, 2), (4, 1, 2, 2)]","[40.0, 72.0, 40.0]","[T2, T2, Ct, T1]",ID,4.0,True,0.0,None,2.0
3,,,Ludwig Senfl,"Ave, maria",0.0,"[1/1.0, 10/3.0, 20/1.0]","[P5, P-4]","[0.0, 76.0, 152.0]","[(5, 1, 2, 2), (4, 1, 2, 2)]","[76.0, 76.0]","[T2, T2, T1]",PEN,3.0,True,0.0,None,2.0
5,,,Ludwig Senfl,"Ave, maria",0.0,"[1/1.0, 6/1.0, 10/3.0, 15/3.0]","[P5, P1, P-11]","[0.0, 40.0, 76.0, 116.0]","[(5, 1, 2, 2), (4, 1, 2, 2)]","[40.0, 36.0, 40.0]","[T2, T2, T2, B]",ID,4.0,True,0.0,None,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,NaN,NaN,Josquin Des Prés,Ave Maria,140.0,"[18/3.0, 20/3.0, 22/3.0]","[P1, P-8]","[140.0, 156.0, 172.0]","[(1, 1, 2, 2)]","[16.0, 16.0]","[Altus, Tenor, Bassus]",PEN,3.0,False,0.0,None,0.0
108,NaN,NaN,Josquin Des Prés,Ave Maria,428.0,"[54/3.0, 55/1.0, 59/3.0, 60/1.0]","[P-5, P-4, P-5]","[428.0, 432.0, 468.0, 472.0]","[(4, -2, 2, 2)]","[4.0, 36.0, 4.0]","[[Superius], Altus, Tenor, Bassus]",ID,4.0,False,0.0,None,0.0
109,NaN,NaN,Josquin Des Prés,Ave Maria,508.0,"[64/3.0, 66/3.0, 67/3.0, 69/3.0]","[P-5, P-4, P-5]","[508.0, 524.0, 532.0, 548.0]","[(2, -3, 2, -3)]","[16.0, 8.0, 16.0]","[[Superius], Altus, Tenor, Bassus]",ID,4.0,False,0.0,None,0.0
111,NaN,NaN,Josquin Des Prés,Ave Maria,748.0,"[94/1.0, 94/2.0, 98/1.0, 98/2.0]","[P-5, P5, P-5]","[748.0, 752.0, 796.0, 800.0]","[(1, 1, 2, 1)]","[4.0, 44.0, 4.0]","[[Superius], Tenor, [Superius], Tenor]",ID,4.0,False,0.0,None,0.0


In [71]:
# filter for maximum time interval between entries
final['pen_increment'] = final['Time_Entry_Intervals'].apply(set).apply(sum)
final = final[final['pen_increment'] < 40].copy()


In [36]:
# remove a particular piece
# final = final[final['Title'] != 'Veni sponsa Christi'].copy()


In [70]:
len(final)

78

## B. Network Visualization with Louvain Communities

* This will only make sense for a corpus!

## Add Combined Data for Melodic, Time and Soggetti as "Words" for Network Edges

* We combine the lists of time and melodic intervals of entries as strings (thus a kind of word that uniquely identifies that pattern)
* We do the same for soggetti, combining the melodic intervals themselves as a string
* These are in turn assembled in various combinations:  TINT, MINT, and SOG, also as strings

In [72]:
final["MINT"] = final["Melodic_Entry_Intervals"].apply(joiner)
final["TINT"] = final["Time_Entry_Intervals"].apply(joiner)

final['SOG'] = final['Soggetti'].apply(clean_melody_new)
final['ALL'] = final["MINT"] + '_' + final["TINT"] + '_' + final['SOG']

final["ALL_INT"] = final["MINT"] + '_' + final["TINT"]
final["MINT_SOG"] = final["MINT"] + '_' + final["SOG"]
final["TINT_SOG"] = final["TINT"] + '_' + final["SOG"]
final.head()

,Validation,Comments,Composer,Title,First_Offset,Measures_Beats,Melodic_Entry_Intervals,Offsets,Soggetti,Time_Entry_Intervals,...,Parallel_Voice,Count_Non_Overlaps,pen_increment,MINT,TINT,SOG,ALL,ALL_INT,MINT_SOG,TINT_SOG
1,,,Ludwig Senfl,"Ave, maria",0.0,"[1/1.0, 3/3.0, 6/1.0]","[M9, P-5]","[0.0, 20.0, 40.0]","[(5, 1, 2, 2), (4, 1, 2, 2)]","[20.0, 20.0]",...,None,0.0,20.0,M9_P-5,20.0_20.0,5_1_2_2,M9_P-5_20.0_20.0_5_1_2_2,M9_P-5_20.0_20.0,M9_P-5_5_1_2_2,20.0_20.0_5_1_2_2
21,,,Ludwig Senfl,"Ave, maria",20.0,"[3/3.0, 8/1.0, 12/3.0]","[P1, P1]","[20.0, 56.0, 92.0]","[(5, 1, 2, 2), (4, 1, 2, 2)]","[36.0, 36.0]",...,None,2.0,36.0,P1_P1,36.0_36.0,5_1_2_2,P1_P1_36.0_36.0_5_1_2_2,P1_P1_36.0_36.0,P1_P1_5_1_2_2,36.0_36.0_5_1_2_2
22,,,Ludwig Senfl,"Ave, maria",20.0,"[3/3.0, 6/1.0, 8/1.0, 10/3.0]","[P-5, P5, P-5]","[20.0, 40.0, 56.0, 76.0]","[(5, 1, 2, 2), (4, 1, 2, 2)]","[20.0, 16.0, 20.0]",...,None,0.0,36.0,P-5_P5_P-5,20.0_16.0_20.0,5_1_2_2,P-5_P5_P-5_20.0_16.0_20.0_5_1_2_2,P-5_P5_P-5_20.0_16.0_20.0,P-5_P5_P-5_5_1_2_2,20.0_16.0_20.0_5_1_2_2
24,,,Ludwig Senfl,"Ave, maria",40.0,"[6/1.0, 10/3.0, 15/1.0]","[P1, M2]","[40.0, 76.0, 112.0]","[(5, 1, 2, 2), (4, 1, 2, 2)]","[36.0, 36.0]",...,None,1.0,36.0,P1_M2,36.0_36.0,5_1_2_2,P1_M2_36.0_36.0_5_1_2_2,P1_M2_36.0_36.0,P1_M2_5_1_2_2,36.0_36.0_5_1_2_2
26,,,Ludwig Senfl,"Ave, maria",40.0,"[6/1.0, 10/3.0, 15/1.0, 19/3.0]","[P1, M2, P4]","[40.0, 76.0, 112.0, 148.0]","[(5, 1, 2, 2), (4, 1, 2, 2)]","[36.0, 36.0, 36.0]",...,None,1.0,36.0,P1_M2_P4,36.0_36.0_36.0,5_1_2_2,P1_M2_P4_36.0_36.0_36.0_5_1_2_2,P1_M2_P4_36.0_36.0_36.0,P1_M2_P4_5_1_2_2,36.0_36.0_36.0_5_1_2_2


### Make Groups, Based on Selected Criteria:

* MINT
* TINT
* SOG
* Or any of the combinations:
    * ALL (MINT + TINT + Soggetti)
    * ALL_INT (MINT + TINT)
    * MINT_SOG (MINT + Soggetti)
    * TINT_SOG (TINT + Soggetti)

In [73]:

output_grouped = final.groupby(['Title'])['MINT'].apply(list).reset_index()
output_grouped


,Title,MINT
0,Ave Maria,"[P-8_P1_P-8, P-8_P1, P1_P-8, P-8_P1_P-8, P-8_P..."
1,"Ave, maria","[M9_P-5, P1_P1, P-5_P5_P-5, P1_M2, P1_M2_P4, P..."


### Group and Find Unique Pairs

- the pairing must be applied to columns that have more than one value, like Title, since these form the edges

In [74]:
pairs = output_grouped.MINT.apply(lambda x: list(combinations(x, 2)))
pairs2 = pairs.explode().dropna()
unique_pairs = pairs.explode().dropna().unique()

In [75]:
unique_pairs

array([('P-8_P1_P-8', 'P-8_P1'), ('P-8_P1_P-8', 'P1_P-8'),
       ('P-8_P1_P-8', 'P-8_P1_P-8'), ('P-8_P1_P-8', 'P-5_P-4_P-5'),
       ('P-8_P1', 'P1_P-8'), ('P-8_P1', 'P-8_P1_P-8'),
       ('P-8_P1', 'P-8_P1'), ('P-8_P1', 'P-5_P-4_P-5'),
       ('P1_P-8', 'P-8_P1_P-8'), ('P1_P-8', 'P-8_P1'),
       ('P1_P-8', 'P1_P-8'), ('P1_P-8', 'P-5_P-4_P-5'),
       ('M9_P-5', 'P1_P1'), ('M9_P-5', 'P-5_P5_P-5'), ('M9_P-5', 'P1_M2'),
       ('M9_P-5', 'P1_M2_P4'), ('M9_P-5', 'P5_P-5_P5'),
       ('M9_P-5', 'P-5_P5_P-4'), ('M9_P-5', 'M2_P4'),
       ('M9_P-5', 'P-12_P15_P-8'), ('M9_P-5', 'P1_P-5_P-8'),
       ('M9_P-5', 'P1_P-12_P12'), ('M9_P-5', 'P-8_P12'),
       ('M9_P-5', 'P15_P1_P-8'), ('M9_P-5', 'P15_P1'),
       ('M9_P-5', 'P1_P-8'), ('M9_P-5', 'P15_P1_P-15'),
       ('M9_P-5', 'P-8_P12_P1'), ('M9_P-5', 'P12_P1'),
       ('M9_P-5', 'P-12_P12'), ('M9_P-5', 'P-5_P5_P-12'),
       ('M9_P-5', 'P-12_P12_P-12'), ('M9_P-5', 'P12_P-12'),
       ('M9_P-5', 'P-12_P8'), ('P1_P1', 'P-5_P5_P-5'), ('P1_P1',

In [76]:
pd.Series(unique_pairs).isna().sum()

0

In [77]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G

def create_node_html(node: str, source_df: pd.DataFrame, node_col: str):
    rows = source_df.loc[source_df[node_col] == node].itertuples()
    
    html_lis = []
    
    for r in rows:
        html_lis.append(f"""<li>Composer: {r.Composer}<br>
                                Title: {r.Title}<br>
                                Offset: {r.First_Offset}</li>"""
                       )
        
    html_ul = f"""<ul>{''.join(html_lis)}</ul>"""
        
    return html_ul


def add_nodes_from_edgelist(edge_list: list, 
                               source_df: pd.DataFrame, 
                               graph: nx.Graph,
                               node_col: str):
    
    graph = deepcopy(graph)
    
    node_list = pd.Series(edge_list).apply(pd.Series).stack().unique()
    
    for n in node_list:
        graph.add_node(n, title=create_node_html(n, source_df, node_col))
        
    return graph

In [78]:
def choose_network(df, chosen_word, file_name):
    
    output_grouped = df.groupby(['Title'])[chosen_word].apply(list).reset_index()
    pairs = output_grouped[chosen_word].apply(lambda x: list(combinations(x, 2)))
    pairs2 = pairs.explode().dropna()
    unique_pairs = pairs.explode().dropna().unique()


    pyvis_graph = Network(notebook=True, width="1800", height="1400", bgcolor="black", font_color="white")
    G = nx.Graph()
    
    try:
        G = add_nodes_from_edgelist(edge_list=unique_pairs, source_df=final, graph=G, node_col=chosen_word)
    except Exception as e:
        print(e)
    
    
    G.add_edges_from(unique_pairs)
    G = add_communities(G)
    pyvis_graph.from_nx(G)
    pyvis_graph.show(file_name)

In [79]:
# Find your rendered network in the Jupyter Hub file list!

choose_network(final, 'MINT', 'Ave_mint_ID_PEN.html')